# **PROBLEMS**


**Problem 1: Parsing 2013/2014 needs assessment data into current fields**

300+ sites were initially identified as part of an informal needs assessment conducted by our Rwanda field staff over 2013 and 2014. We collected much of the same data that we do now, but in a less structured format. When we created the database in 2017, we loaded all the old needs assessment into the general comments field on each bridge record (column L in this table), just to keep as a reference. We now believe that loading this data into our standard project assessments would be a benefit to future needs assessment work, both field-based and remote. We would love to see the needs assessment data from Column L parsed into the corresponding columns after it (M+), with a new row if there is existing data in those columns. If no corresponding column is obvious, we'd like to see new columns created for any extraneous data that is common to the old format. 

**Problem 2: Predicting which sites will be technically rejected in future engineering reviews**

Any sites with a "Yes" in the column AQ have undergone a full technical review, and of those, the Stage (column L) can be considered to be correct. Any sites with a "Yes" in Column AQ have not undergone a full technical review, and the Stage is based on the assessor's initial estimate as to whether the site was technically feasible or not. We want to know if we can use the sites that have been reviewed to understand which of the sites that haven't yet been reviewed are likely to be rejected by the senior engineering team. Any of the data can be used, but our guess is that Estimated Span, Height Differential Between Banks, Created By, and Flag for Rejection are likely to be the most reliable predictors.

---



# **Exploratory Analysis**

In [ ]:
# Imports
import numpy as np
import pandas as pd
from datetime import date



In [ ]:
# Load data
B2P_2020_Oct = pd.read_csv("https://raw.githubusercontent.com/JonRivera/Bridges_Prosperity_Labs28_Project/main/Data/B2P%20Dataset_2020.10.csv")

In [ ]:
B2P_2020_Oct.shape

(1518, 43)

In [ ]:
B2P_2020_June = pd.read_csv("https://raw.githubusercontent.com/JonRivera/Bridges_Prosperity_Labs28/main/Data/B2P%20Rwanda%20Site%20Assessment%20Data_2020.06.03.csv")

In [ ]:
B2P_2020_June.head()

,Country,Province,District,Sector,Cell,Bridge Site Name,Project Stage,Project Sub-Stage,Project Code,Bridge Type,Span (m),GPS (Latitude),GPS (Longitude),Individuals Directly Served,Form: Form Name,Community Served 1,Community Served 2,Community Served 3,Community Served 4,Community Served 5,Community Served 6,Community Served 7,Community Served 8,Community Served 9,Community Served 10,CaseSafeID Form,Bridge Opportunity: Opportunity ID
0,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Buzi,Rejected,Technical,1014107,Suspended,NaN,-2.420560,28.966200,NaN,Project Assessment - 2018.10.29,Buzi,Kabuga,Kagarama,Gacyamo,Gasheke,NaN,NaN,NaN,NaN,NaN,a1if1000002e51bAAA,006f100000d1fk1
1,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Kamigisha,Rejected,Technical,1014106,Suspended,NaN,-2.424860,28.957260,NaN,Project Assessment - 2018.10.29,Kabuga,Buzi,Gacyamo - Gasheke,Kagarama,Kanoga,NaN,NaN,NaN,NaN,NaN,a1if1000002e51WAAQ,006f100000d1fjw
2,Rwanda,Northern Province,Rulindo,Buyoga,Gahororo-Gipfundo,Gipfundo,Rejected,Technical,1007651,Suspended,8.0,-1.720530,30.081240,NaN,Project Assessment - 2018.8.11,Gapfundo,Rusabira ( Gicumbi),Merezo,Minanire,Karutongo,NaN,NaN,NaN,NaN,NaN,a1if10000025nz8AAA,006f100000a86I3
3,Rwanda,Northern Province,Gicumbi,Kageyo,Kabuga/gatobotobo,Nyarubande,Rejected,NaN,1012493,Other,NaN,-1.655950,30.078840,NaN,Project Assessment - 2018.8.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a1if10000025nzDAAQ,006f100000cPpL8
4,Rwanda,Southern Province,Kamonyi,Kayenzi,Kirwa,Gisizi,Identified,Requested,1014318,Suspended,NaN,-1.870868,29.877686,NaN,Project Assessment - 2018.11.15,Gisizi,Nyabitare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a1if1000002gMwRAAU,006f100000eescb


In [ ]:
rejection = B2P_2020_Oct[(B2P_2020_Oct['Flag for Rejection']=='Yes') & (B2P_2020_Oct['Senior Engineering Review Conducted'] == "Yes")]

In [ ]:
accepted = B2P_2020_Oct[(B2P_2020_Oct['Flag for Rejection']=='No') & (B2P_2020_Oct['Senior Engineering Review Conducted'] == "Yes")]

In [ ]:
#Redundant features:
# 'Bridge Opportunity: GPS (Latitude)',
# 'Bridge Opportunity: GPS (Longitude)',

In [ ]:
rejection = rejection[['Bridge Name', 'Bridge Opportunity: Project Code',
       'Bridge Opportunity: Bridge Type', 'Bridge Opportunity: Span (m)',
       'Bridge Opportunity: Individuals Directly Served','Bridge Opportunity: Comments',
       'Form: Form Name', 
       'Proposed Bridge Location (GPS) (Latitude)',
       'Proposed Bridge Location (GPS) (Longitude)', 'Current crossing method',
       'Nearest all-weather crossing point', 'Days per year river is flooded',
       'Flood duration during rainy season', 'Market access blocked by river',
       'Education access blocked by river', 'Health access blocked by river',
       'Other access blocked by river', 'Primary occupations',
       'Primary crops grown', 'River crossing deaths in last 3 years',
       'River crossing injuries in last 3 years', 'Incident descriptions',
       '4WD Accessibility', 'Name of nearest city',
       'Name of nearest paved or sealed road', 'Bridge classification',
       'Flag for Rejection', 'Rejection Reason', 'Bridge Type',
       'Estimated span (m)', 'Height differential between banks',
       'Senior Engineering Review Conducted']]

In [ ]:
rejection.head(2)

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Bridge Opportunity: Comments,Form: Form Name,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Incident descriptions,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Senior Engineering Review Conducted
0,Bukinga,1009317,Suspended Bridge,NaN,NaN,Steep slopes to deal with for anchors,Project Assessment - 2018.10.10,-2.760833,29.488056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huye,NaN,NaN,Yes,There is a planned vehicular road with a vehicular bridge on this site,NaN,NaN,NaN,Yes
5,Nyakabuye - Mwogo,1007507,Suspension Bridge,118.0,2725.0,"Not Duplicate 3000-6000 people directly served, Elevation:1523m, Cell:Gasave-Rwesero, Bridge would connecting Nyamagabe-Ruhango, Injuries/Death-Many peoples injured while trying to cross the river/ More than 4peoples died while trying to cross the river Cross river on a normal day-50-150 people, Nearby city centers--Masizi -Buhanda, Crossing River now-Swim / BarogaBoat / Bakoresha ubwato, Impossible/Dangerous to cross the river-9-12 months / Hagati y?amezi icyenda na cuminabiri,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital-2-6 hours,Hours walking to reach the Health Center-1-2 hours,Hours of walking to reach the market-2-6 hours,Hours walking to reach Primary School-0.5-1 hours,Hours walking to reach Secondary School-0.5-1 hours,Hours walking to reach the Church-0.5-1 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone-Available / birahaboneka,Timber-Available / birahaboneka,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 20m.-The proposed bridge span is approximately 100m.-The level difference between two banks is 0.1m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is sufficient-There is no confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is showing a sign of erosion and it is located between two flat areas.-The soil from the site is silt for both sides. <<On 25th March, 2019 _In the beginning of scaling, this site was rejected by the assessor thinking that it will be a very long span bridge out of B2P standard but having the plan to survey Nyakabuye - Nkomane over a joining river""Nkomane"", this Nyakabuye - Mwogo site over Mwogo river was surveyed and found to be technically feasible. >>>",Project Assessment - 2018.9.4,-2.233369,29.575727,none,vehicular is in two hours walk away,365.0,2day,People from Musange Sector particularly from Gasave cell are b

# **Extracting Data Inside Comments and Converting Data Into Features for Data Frame [TOKENS APPROACH]**

In [ ]:
B2P_2020_Oct_Comments = B2P_2020_Oct[["Bridge Opportunity: Project Code",'Bridge Opportunity: Comments']]

In [ ]:
#Count how many null values in the comments
B2P_2020_Oct_Comments.isna().sum()

Bridge Opportunity: Project Code    21  
Bridge Opportunity: Comments        1124
dtype: int64

In [ ]:
#We have 1497 observation but only 394 of these observations actually have commments
B2P_2020_Oct_Comments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1518 entries, 0 to 1517
Data columns (total 2 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Bridge Opportunity: Project Code  1497 non-null   object
 1   Bridge Opportunity: Comments      394 non-null    object
dtypes: object(2)
memory usage: 23.8+ KB


In [ ]:
#Set display options so that u can see everything inside a cell
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
#Exclude null rows
B2P_2020_Oct_Comments = B2P_2020_Oct_Comments[B2P_2020_Oct_Comments['Bridge Opportunity: Comments'].notnull()]

In [ ]:
B2P_2020_Oct_Comments.head(10)

,Bridge Opportunity: Project Code,Bridge Opportunity: Comments
0,1009317,Steep slopes to deal with for anchors
2,1007374,"Likely piloting steel decking on this bridge. 6000-10000 people directly served, Elevation:1630m, Cell:Karambi-Gasumba, Injuries/Death-many peoples injured while trying to cross the river/ 2 peoples died while trying to cross the river Cross river on a normal day->600 people, Nearby city centers--Karambi -Nyamagabe, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-6-9 months / Hagati y?amezi atandatu n?icyenda,Travel to nearest safe bridge/river crossing->3 hours / Hejuru y?amasaha 3,Hours walking to reach the Hospital-0.5-1 hours,Hours walking to reach the Health Center-0.5 hours,Hours of walking to reach the market-0.5 hours,Hours walking to reach Primary School-0.5 hours,Hours walking to reach Secondary School-0.5 hours,Hours walking to reach the Church-0.5 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone-Available / birahaboneka,Timber-Available / birahaboneka,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Private landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 10m.-The proposed bridge span is approximately 70m.-The level difference between two banks is 0.06m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is sufficient-There is no confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is not showing any sign of erosion and it is located between flat areas.-The soil from the site is silt for both sides."
3,1007466,"3000-6000 people directly served, Elevation:2101m, Cell:Rugogwe, Injuries/Death-Many peoples injured while trying to cross the river/ No person died while trying to cross the river Cross river on a normal day-300-600 people, Nearby city centers--Kizimyamuriro, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-3-6 months / Hagati y?amezi atatu n?atandatu,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital->6 hours,Hours walking to reach the Health Center-0.5-1 hours,Hours of walking to reach the market-0.5-1 hours,Hours walking to reach Primary School-0.5-1 hours,Hours walking to reach Secondary School-0.5-1 hours,Hours walking to reach the Church-0.5-1 hours,Land within 50m of river bank-hilly / umusozi,Soil-hard rock/ Urutare,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone-Available / birahaboneka,Timber-Available / birahaboneka,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 1m.-The proposed bridge span is approximately 55m.-The level difference between

In [ ]:
B2P_2020_Oct_Comments.columns

Index(['Bridge Opportunity: Project Code', 'Bridge Opportunity: Comments'], dtype='object')

In [ ]:
#Create tokenize function, splits based "/"
import numpy as np
import re

def tokenize(text):
    """Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    
    text = str(text)
    res = re.split(',', text) 
    return res

In [ ]:
storage = {}
def tokenize2(code,text):
    global storage
    """Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        updates the dictionary based on bridge code(key), the tokenization represents the values  
        
    """
    text = str(text)
    res = re.split('/', text) 
    storage[code] = res

In [ ]:

B2P_2020_Oct_Comments['tokens']= B2P_2020_Oct_Comments['Bridge Opportunity: Comments'].apply(tokenize)

In [ ]:
for x,y in zip(B2P_2020_Oct_Comments['Bridge Opportunity: Project Code'], B2P_2020_Oct_Comments['Bridge Opportunity: Comments']):
  tokenize2(x,y)


In [ ]:
# Converting Items in Storage(Splitted Comments) Into DataFrame To Create  A 2013-2014 Based Data Set
B2P_2013_2014 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in storage.items() ])).T


In [ ]:
B2P_2013_2014.info()

<class 'pandas.core.frame.DataFrame'>
Index: 394 entries, 1009317 to 1015561
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       394 non-null    object
 1   1       257 non-null    object
 2   2       254 non-null    object
 3   3       254 non-null    object
 4   4       253 non-null    object
 5   5       253 non-null    object
 6   6       253 non-null    object
 7   7       253 non-null    object
 8   8       253 non-null    object
 9   9       253 non-null    object
 10  10      252 non-null    object
 11  11      252 non-null    object
 12  12      252 non-null    object
 13  13      252 non-null    object
 14  14      252 non-null    object
 15  15      252 non-null    object
 16  16      252 non-null    object
 17  17      252 non-null    object
 18  18      252 non-null    object
 19  19      252 non-null    object
 20  20      250 non-null    object
 21  21      176 non-null    object
 22  22      43 non-null  

In [ ]:
B2P_2013_2014.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
1009317,Steep slopes to deal with for anchors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007374,"Likely piloting steel decking on this bridge. 6000-10000 people directly served, Elevation:1630m, Cell:Karambi-Gasumba, Injuries",Death-many peoples injured while trying to cross the river,"2 peoples died while trying to cross the river Cross river on a normal day->600 people, Nearby city centers--Karambi -Nyamagabe, Crossing River now-Simple timber bridge","hari uduti, Impossible",Dangerous to cross the river-6-9 months,"Hagati y?amezi atandatu n?icyenda,Travel to nearest safe bridge",river crossing->3 hours,"Hejuru y?amasaha 3,Hours walking to reach the Hospital-0.5-1 hours,Hours walking to reach the Health Center-0.5 hours,Hours of walking to reach the market-0.5 hours,Hours walking to reach Primary School-0.5 hours,Hours walking to reach Secondary School-0.5 hours,Hours walking to reach the Church-0.5 hours,Land within 50m of river bank-flat (flood plain)","ikibaya,Soil-Silt,Sand-Available","birahaboneka,Gravel-Not available on site, but local government can provide","Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone-Available","birahaboneka,Timber-Available","birahaboneka,Stone provided by-Sector","umurenge,Sand Provided by-Sector","umurenge,Gravel provided by-District","Akarere,Timber provided by-District","Akarere,Cement provided by-Can not be provided at the time","Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time","Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Private landowner","Umuturage, Land ownership permission-Yes","Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 10m.-The proposed bridge span is approximately 70m.-The level difference between two banks is 0.06m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is sufficient-There is no confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is not showing any sign of erosion and it is located between flat areas.-The soil from the site is silt for both sides.",NaN,NaN,NaN


In [ ]:
#Extraction function tghat filters for key information
#Function will place certain info/data inside a dictionary
#Using regex/filtering techniques I will search for info and save into dictionary
#Dictionary keys will correspond to column features in 2018 data set
#Desired Structures => => => [1] are dummy values, the goal is for them to have values corresponding to the key
dictionary0 = {'Bridge Name':[1], #
'Bridge Opportunity: Bridge Type':[1],
'Bridge Opportunity: Span (m)':[1],
'Bridge Opportunity: Individuals Directly Served':[11],
'Form: Form Name':[1], 
'Proposed Bridge Location (GPS) (Latitude)':[1],
'Proposed Bridge Location (GPS) (Longitude)':[1],
'Current crossing method':[1],
'Nearest all-weather crossing point':[1],
'Days per year river is flooded':[1],
'Flood duration during rainy season':[1],
'Market access blocked by river':[1],
'Education access blocked by river':[1], 
'Health access blocked by river':[1],
'Other access blocked by river':[1], 
'Primary occupations':[1],
'Primary crops grown':[1], #
'River crossing deaths in last 3 years':[1],
'River crossing injuries in last 3 years':[1],
 'Incident descriptions':[1],
'4WD Accessibility':[1], 
'Name of nearest city':[1],
'Name of nearest paved or sealed road':[1], 
'Bridge classification':[11],
'Flag for Rejection':[1],
'Rejection Reason':[1],
'Bridge Type':[1],
'Estimated span (m)':[1],
'Height differential between banks':[1],
'Senior Engineering Review Conducted':[1],
'Elevation':[1],
'people_served':[1]} #kew words number_range people
#key words to search for
# hour,
# months

In [ ]:
B2P_2013_2014_V2 = pd.DataFrame(dictionary0)

In [ ]:
B2P_2013_2014_V2.head()

,Bridge Name,Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Form: Form Name,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Incident descriptions,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Senior Engineering Review Conducted,Elevation,people_served
0,1,1,1,11,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,11,1,1,1,1,1,1,1,1


# **Working with Fuzzy Wuzzy**

In [ ]:
#Will attempt to us fuzzywuzzy to find certain information in the comments section

!pip install fuzzywuzzy

# **Testing Regular Expressions**

In [ ]:
import re
# Searching for numbers -> I want to look for people served -> 
find_people_range_numbers = re.findall(r'(^.*people+[ ])([0-9]+-[0-9]+)','people 2000-6000')
#Search for name John
find_people = re.findall(r'^.*John.*$','John')
find_dangerous = re.findall(r'\bDangerous\b','Dangerous dogs')
#Searching for Dangerous
find_123 = re.findall(r'^.*?\b(one|two|three)\b.*$','duck one\,two,three')
#Find number after a somewords
find_number = re.findall(r"([a-zA-Z]*[ ])+([0-9]+)",'wordsa 3243')
#Search for first and last name
find_first_last = re.findall(r"([a-zA-Z]+[ ])*([a-zA-Z]+)",'1221 John Rivera 213 dfsf fsdf')
#searching for range number followed by people - represented number of people served
find_people_served = re.findall(r'(^[0-9]+-[0-9]+[ ])(people)','2000-3000 people')
#searching for elevation followed by number: ex) Elevation:2101m
elavation = re.findall(r'(^.*Elevation+:)([0-9]+)','Elevation:2101')
# Combinding multiple regex together
re_list = [r'[0-9]+',r'^.*John.*$']
generic_re = re.compile( '|'.join( re_list))
print('find_people_range_numbers:',find_people_range_numbers,
      'find_123:',find_123,
      'find_number_after_words:',find_number,
      'find_people_served:',find_people_served)
print('combind multiple regex',generic_re.findall('John 100- 12323 and then'))

find_people_range_numbers: [('people ', '2000-6000')] find_123: ['one'] find_number_after_words: [('wordsa ', '3243')] find_people_served: [('2000-3000 ', 'people')]
combind multiple regex ['John 100- 12323 and then']


In [ ]:
print('elevation:',elavation)

elevation: [('Elevation:', '2101')]


# **Testing Regular Expression On A Comment**

In [ ]:
test_comment = pd.DataFrame({'Comment1':['3000-6000 people directly served, Elevation:2101m, Cell:Rugogwe, Injuries/Death-Many peoples injured while trying to cross the river/ No person died while trying to cross the river Cross river on a normal day-300-600 people, Nearby city centers--Kizimyamuriro, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-3-6 months / Hagati y?amezi atatu n?atandatu,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital->6 hours,Hours walking to reach the Health Center-0.5-1 hours,Hours of walking to reach the market-0.5-1 hours,Hours walking to reach Primary School-0.5-1 hours,Hours walking to reach Secondary School-0.5-1 hours,Hours walking to reach the Church-0.5-1 hours,Land within 50m of river bank-hilly / umusozi,Soil-hard rock/ Urutare,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone-Available / birahaboneka,Timber-Available / birahaboneka,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 1m.-The proposed bridge span is approximately 55m.-The level difference between two banks is 0.2m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is sufficient-There is no confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is not showing any sign of erosion and it is located between elevated areas.-The soil from the site is hard rock for one side and silt for the other side']})

In [ ]:
test_comment

,Comment1
0,"3000-6000 people directly served, Elevation:2101m, Cell:Rugogwe, Injuries/Death-Many peoples injured while trying to cross the river/ No person died while trying to cross the river Cross river on a normal day-300-600 people, Nearby city centers--Kizimyamuriro, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-3-6 months / Hagati y?amezi atatu n?atandatu,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital->6 hours,Hours walking to reach the Health Center-0.5-1 hours,Hours of walking to reach the market-0.5-1 hours,Hours walking to reach Primary School-0.5-1 hours,Hours walking to reach Secondary School-0.5-1 hours,Hours walking to reach the Church-0.5-1 hours,Land within 50m of river bank-hilly / umusozi,Soil-hard rock/ Urutare,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,Stone-Available / birahaboneka,Timber-Available / birahaboneka,Stone provided by-Sector/ umurenge,Sand Provided by-Sector/ umurenge,Gravel provided by-District/ Akarere,Timber provided by-District/ Akarere,Cement provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Reinforcement steel provided by-Can not be provided at the time / Ntaruhare rwamenyekana muri aka kanya.,Land ownership-Government of Rwanda / Guverinoma y? u RwandaPrivate landowner/ Umuturage, Land ownership permission-Yes / Yego, General Comments-The proposed bridge location is near the traditional crossing point at a distance of approximately 1m.-The proposed bridge span is approximately 55m.-The level difference between two banks is 0.2m.-The space for foundation is sufficient-The free board between the lowest point of the proposed bridge and the highest flood level is sufficient-There is no confluence area near the place.-The river bed at the site is stable, there is no possibility of erosion.-The river bank of the site is not showing any sign of erosion and it is located between elevated areas.-The soil from the site is hard rock for one side and silt for the other side"


In [ ]:
find_people_range_numbers = re.findall(r'(^.*people+[ ])([0-9]+-[0-9]+)','people 2000-6000')

In [ ]:
test_comment['Comment1']

0    3000-6000 people directly served, Elevation:2101m, Cell:Rugogwe, Injuries/Death-Many peoples injured while trying to cross the river/ No person died while trying to cross the river Cross river on a normal day-300-600 people, Nearby city centers--Kizimyamuriro, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-3-6 months / Hagati y?amezi atatu n?atandatu,Travel to nearest safe bridge/river crossing-2-3 hours / Hagati yamasaha 2 n?amasaha 3,Hours walking to reach the Hospital->6 hours,Hours walking to reach the Health Center-0.5-1 hours,Hours of walking to reach the market-0.5-1 hours,Hours walking to reach Primary School-0.5-1 hours,Hours walking to reach Secondary School-0.5-1 hours,Hours walking to reach the Church-0.5-1 hours,Land within 50m of river bank-hilly / umusozi,Soil-hard rock/ Urutare,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka ariko inzego z?ubuyobozi zabitanga,

In [ ]:
storage2 = {} # This line resets the dictionary so only run it once, unless u want to reset the dictionary
def extractor(code,text):
  global storage2
  text = str(text)
  code = str(code)
  find_people_served = re.findall('(^[0-9]+-[0-9]+[ ])(people)',text)
  storage2[code] = find_people_served
  return storage2


# **B2P_2020_Oct_Comments: Extracting Number of People Served:**



In [ ]:
#Extracting People Served by The Bridge Data
for code, comment in zip(B2P_2020_Oct_Comments['Bridge Opportunity: Project Code'],B2P_2020_Oct_Comments['Bridge Opportunity: Comments']):
  extractor(code,comment)

In [ ]:
#The code above filters for people directly served implementing the extractor function, saves everything into a dictionary, where the key represents a bridge code
#Empty list represents data was not found in the comment section
len(storage2),storage2

(394,
 {'1007327': [('500-1000 ', 'people')],
  '1007328': [('500-1000 ', 'people')],
  '1007329': [('1000-3000 ', 'people')],
  '1007330': [('500-1000 ', 'people')],
  '1007331': [],
  '1007332': [('6000-10000 ', 'people')],
  '1007334': [],
  '1007335': [],
  '1007336': [],
  '1007338': [('1000-3000 ', 'people')],
  '1007339': [('3000-6000 ', 'people')],
  '1007340': [('500-1000 ', 'people')],
  '1007342': [('3000-6000 ', 'people')],
  '1007343': [('3000-6000 ', 'people')],
  '1007344': [('3000-6000 ', 'people')],
  '1007345': [('3000-6000 ', 'people')],
  '1007347': [('500-1000 ', 'people')],
  '1007348': [('3000-6000 ', 'people')],
  '1007349': [('3000-6000 ', 'people')],
  '1007350': [('500-1000 ', 'people')],
  '1007352': [('500-1000 ', 'people')],
  '1007354': [('1000-3000 ', 'people')],
  '1007356': [('1000-3000 ', 'people')],
  '1007357': [('1000-3000 ', 'people')],
  '1007360': [('3000-6000 ', 'people')],
  '1007364': [('6000-10000 ', 'people')],
  '1007366': [('3000-6000 ', 

# **B2P_2020_Oct_Comments Extracting Elevation** (Builds on Top Work, Dictionary Gets Updated)


In [ ]:
def extractor1(code,text):
  global storage2
  text = str(text)
  code = str(code)
  elevation = re.findall(r'(^.*Elevation+:)([0-9]+)',text)
  storage2[code]+=elevation
  return storage2

In [ ]:
for code, comment in zip(B2P_2020_Oct_Comments['Bridge Opportunity: Project Code'],B2P_2020_Oct_Comments['Bridge Opportunity: Comments']):
  extractor1(code,comment)

In [ ]:
storage2

{'1007327': [('500-1000 ', 'people'),
  ('500-1000 people directly served, Elevation:', '1325'),
  ('500-1000 people directly served, Elevation:', '1325')],
 '1007328': [('500-1000 ', 'people'),
  ('500-1000 people directly served, Elevation:', '1329'),
  ('500-1000 people directly served, Elevation:', '1329')],
 '1007329': [('1000-3000 ', 'people'),
  ('1000-3000 people directly served, Elevation:', '1333'),
  ('1000-3000 people directly served, Elevation:', '1333')],
 '1007330': [('500-1000 ', 'people'),
  ('500-1000 people directly served, Elevation:', '1326'),
  ('500-1000 people directly served, Elevation:', '1326')],
 '1007331': [("BAJ confirms rejection. Review behavior has changed at this location over time. After 2018 rainy season the river flooded much higher than ever before and the required span would be 400+ meters. The range finders could not shoot to the other side. Site is rejected based on being Beyond B2P's current engineering scope. 500-1000 people directly served, E

# **B2P_2020_Oct_Comments: Extracting Cell**


# **B2P_2020_Oct_Comments: Extracting Information on Crossing River**
